Answer a few questions and calculate your fatality risk once contracted COVID-19.

## Load model and data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import pickle
import joblib

import warnings

warnings.simplefilter("ignore")

In [2]:
# Read data and models from pickles
with open("covid19_risk_xgboost.pkl", "rb") as file:
    xgb_best = pickle.load(file)

with open("age_scaler.pkl", "rb") as file:
    age_scaler = pickle.load(file)

with open("docs_scaler.pkl", "rb") as file:
    docs_scaler = pickle.load(file)

with open("beds_scaler.pkl", "rb") as file:
    beds_scaler = pickle.load(file)

with open("medical.pkl", "rb") as file:
    medical = pickle.load(file)

## Take new input

In [3]:
# Function: Take patient info
def patient_info():

    # Demographic info
    age = input("What's your age?")
    sex = input("What's your biological sex? (female: 1, male: 0)")
    country = input("Which country do you live in?")
    docs_per_10k = medical.loc[
        medical["country"].str.contains(country), "docs_per_10k"
    ].item()
    beds_per_10k = medical.loc[
        medical["country"].str.contains(country), "beds_per_10k"
    ].item()

    # Pre-conditions
    preconition = input("Any chronic dieases that you know of? (yes: 1, no: 0)")
    hypertension = input("Do you have hypertension? (yes: 1, no: 0)")
    diabetes = input("Do you have diabetes? (yes: 1, no: 0)")
    heart = input("Do you have heart diseases? (yes: 1, no: 0)")

    # Symptoms
    fever = input("Do you have fever? (yes: 1, no: 0)")
    cough = input("Are you coughing? (yes: 1, no: 0)")
    fatigue = input("Do you feel fatigue? (yes: 1, no: 0)")
    sore_throat = input("Do you have a sore throat? (yes: 1 , no:0)")

    # Save results in a list
    return {
        "sex": int(sex),
        "chronic_disease_binary": int(preconition),
        "beds_per_10k": float(beds_scaler.transform([[beds_per_10k]])),
        "docs_per_10k": float(docs_scaler.transform([[docs_per_10k]])),
        "hypertension": int(hypertension),
        "diabetes": int(diabetes),
        "heart": int(heart),
        "fever": int(fever),
        "cough": int(cough),
        "fatigue": int(fatigue),
        "sore throat": int(sore_throat),
        "age_scaled": float(age_scaler.transform([[age]])),
    }


# Function: Output risk
def risk_predictor():

    # Get patient info
    new_case = patient_info()

    # Convert to useable format
    new_case = pd.DataFrame(new_case, index=[0])

    # Make prediction
    new_prediction = xgb_best.predict_proba(new_case)

    # Print result
    print(
        "Once contracted COVID-19, your mortality risk is {}%.".format(
            round(new_prediction.tolist()[0][1] * 100, 2)
        )
    )

## Try it for yourself

In [ ]:
# Uncomment the function below 
# risk_predictor()